In [6]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from os import path

os.getcwd()

'/home/matta/recipy_maker/Web_Scraping'

In [9]:
MAIN_STRING = "carouselNav__list elementFont__resetList recipeCarousel__list"  # class for main recipe category page navlist
CAT_STRING = "category-page-list-content category-page-list-content__recipe-card karma-main-column"  # Class for category page div
CARDS_STRING = "card__titleLink manual-link-behavior elementFont__title margin-8-bottom"  # Class for most popular recipe cards

def get_links():
    if path.exists('recipe_list_w_img.csv'):
        df = pd.read_csv('recipe_list_w_img.csv')
    else:
        source = requests.get('https://www.allrecipes.com/recipes/').text
        soup = BeautifulSoup(source, 'lxml')
        recipe_list = soup.find('ul', class_=MAIN_STRING)
        links = [a['href'] for a in recipe_list.find_all('a', href=True)]
        pop_recipes = []
        for link in links:
            category = link.split('/')[5]
            recipe_category = requests.get(link).text
            cat_soup = BeautifulSoup(recipe_category, 'lxml')  # category page (cat_soup)
            category_lists = cat_soup.find_all('div', class_=CAT_STRING)  
            for lst in category_lists:
                for a in lst.find_all('a', class_=CARDS_STRING, href=True):
                    if re.search('https://www.allrecipes.com/recipe/.*', a['href']):
                        pop_recipes.append([a['href'], a.div['data-src'], category])
        


        df = pd.DataFrame(pop_recipes, columns=["link","image_url","category"])
        df.to_csv('recipe_list_w_img.csv', index=False)
    return df



In [10]:
df = get_links()
df = df.drop_duplicates()
df.shape
# Why does enumerate return less... 511 vs 659

(660, 3)

In [11]:
# Runs ~2 minutes

recipe_info = []
for index,row in df.iterrows():
    link = row['link']
    image_url = row['image_url']
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'lxml')
    try:
        title = soup.find('h1').text
        description = soup.find('p').text
        rating = soup.find('span',class_="review-star-text visually-hidden").text

        ingredient_list = soup.find('ul',class_ = "ingredients-section")
        ingredients = []
        for ingredient in ingredient_list.find_all('span',class_="ingredients-item-name elementFont__body"):
            ingredients.append(ingredient.text)

        instruction_list = soup.find('ul',class_ = "instructions-section")
        instructions = []
        for instruction in instruction_list.find_all('p'):
            instructions.append(instruction.text)
        
        recipe_info.append([link,title,image_url,description,rating,ingredients,instructions])
    except:
        continue

df2 =  pd.DataFrame(recipe_info, columns=["link","title","image_url","description","rating","ingredients","instructions"])


In [12]:
df2

,link,title,image_url,description,rating,ingredients,instructions
0,https://www.allrecipes.com/recipe/221286/tradi...,Traditional Mexican Guacamole,https://imagesvc.meredithcorp.io/v3/mm/image?u...,This guacamole is great! Whether or not you wa...,Rating: 4.5 stars,"[2 avocados, peeled and pitted , 1 cup chopped...","[Mash avocados in a bowl until creamy., Mix to..."
1,https://www.allrecipes.com/recipe/14811/creamy...,Creamy Dill Cucumber Toasties,https://imagesvc.meredithcorp.io/v3/mm/image?u...,I got this recipe years ago from a friend of a...,Rating: 4.5 stars,"[1 (8 ounce) package cream cheese, softened , ...","[In a medium bowl, mix together cream cheese, ..."
2,https://www.allrecipes.com/recipe/73135/homest...,Homestyle Potato Chips,https://imagesvc.meredithcorp.io/v3/mm/image?u...,Homemade potato chips are fun and easy. Guaran...,Rating: 4.5 stars,"[4 medium potatoes, peeled and sliced paper-th...",[Transfer potato slices to a large bowl of col...
3,https://www.allrecipes.com/recipe/223491/fully...,Fully Loaded Deviled Eggs,https://imagesvc.meredithcorp.io/v3/mm/image?u...,My children and husband go nuts over these dev...,Rating: 4.5 stars,"[12 eggs , 1 teaspoon vegetable oil , 1 small ...",[Place eggs into a large pot and cover with li...
4,https://www.allrecipes.com/recipe/69225/strawb...,Strawberry Bruschetta,https://imagesvc.meredithcorp.io/v3/mm/image?u...,This is a delicious variation of the popular t...,Rating: 4.5 stars,"[24 slices French baguette , 1 tablespoon butt...",[Preheat your oven's broiler. Spread a thin la...
...,...,...,...,...,...,...,...
653,https://www.allrecipes.com/recipe/6773/simple-...,Simple Whole Wheat Bread,https://imagesvc.meredithcorp.io/v3/mm/image?u...,"A great-tasting whole wheat bread recipe, easy...",Rating: 5 stars,[3 cups warm water (110 degrees F/45 degrees C...,"[Mix warm water, yeast, and 1/3 cup honey in a..."
654,https://www.allrecipes.com/recipe/85452/homema...,Homemade Black Bean Veggie Burgers,https://imagesvc.meredithcorp.io/v3/mm/image?u...,"With this black bean burger recipe, you will n...",Rating: 4.5 stars,"[ cooking spray , 1 (16 ounce) can black beans...",[Preheat an outdoor grill for high heat. Light...
655,https://www.allrecipes.com/recipe/9377/roasted...,Roasted Vegetables,https://imagesvc.meredithcorp.io/v3/mm/image?u...,A casserole dish of seasonal vegetables that i...,Rating: 4.5 stars,"[1 small butternut squash, cubed , 2 red bell ...",[Preheat oven to 475 degrees F (245 degrees C)...
656,https://www.allrecipes.com/recipe/94055/best-b...,Best Bean Salad,https://imagesvc.meredithcorp.io/v3/mm/image?u...,This three-bean salad is delicious. I'd turned...,Rating: 4.5 stars,"[1 (15.5 ounce) can garbanzo beans, drained , ...","[Combine garbanzo beans, kidney beans, black b..."


In [13]:
df2.to_pickle('full_data_with_image.pkl')